## Cloud Function Script

##### Note:
##### This cloud function is to embed the recipe name and description

[What is cloud function](https://cloud.google.com/vertex-ai/docs/pipelines/trigger-pubsub)

### Cloud Function

In [ ]:
import functions_framework

from google.cloud import bigquery

def trigger_sql_and_save():
    client = bigquery.Client()
    query = """
    WITH tmp AS (
      SELECT (description || name) as content, id as title
      FROM `brave-watch-414204.RecipeQuery.recipes`
      WHERE _PARTITIONTIME = (
        SELECT MAX(_PARTITIONTIME)
        FROM `brave-watch-414204.RecipeQuery.recipes`
      )
    )
    SELECT *
    FROM ML.GENERATE_EMBEDDING(
      MODEL `RecipeQuery.gecko_model`,
      (SELECT content as content, title as title FROM `tmp`),
      STRUCT(TRUE AS flatten_json_output)
    )
    """

    # Set the destination table and overwrite option
    job_config = bigquery.QueryJobConfig(
        destination="brave-watch-414204.RecipeQuery.embedding_result",
        write_disposition="WRITE_TRUNCATE"  # Overwrites the table
    )

    # Start the query, passing in the extra configuration.
    query_job = client.query(query, job_config=job_config)
    query_job.result()  # Wait for the job to complete.

    return 'Query executed and table overwritten successfully.'



# CloudEvent function to be triggered by an Eventarc Cloud Audit Logging trigger
# Note: this is NOT designed for second-party (Cloud Audit Logs -> Pub/Sub) triggers!
@functions_framework.cloud_event
def hello_auditlog(cloudevent):
    # Print out the CloudEvent's (required) `type` property
    # See https://github.com/cloudevents/spec/blob/v1.0.1/spec.md#type
    print(f"Event type: {cloudevent['type']}")

    # Print out the CloudEvent's (optional) `subject` property
    # See https://github.com/cloudevents/spec/blob/v1.0.1/spec.md#subject
    if 'subject' in cloudevent:
        # CloudEvent objects don't support `get` operations.
        # Use the `in` operator to verify `subject` is present.
        print(f"Subject: {cloudevent['subject']}")

    # Print out details from the `protoPayload`
    # This field encapsulates a Cloud Audit Logging entry
    # See https://cloud.google.com/logging/docs/audit#audit_log_entry_structure

    payload = cloudevent.data.get("protoPayload")
    if payload:
        print(f"API method: {payload.get('methodName')}")
        print(f"Resource name: {payload.get('resourceName')}")
        print(f"Principal: {payload.get('authenticationInfo', dict()).get('principalEmail')}")

    print("Hello >>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    result=trigger_sql_and_save()
    print("Function Finish >>>>>>>>>>>>>>>>>>>>>>>>>>>>")

## requirements.txt

### Add schema.yaml file into your cloud funtion folder